In [112]:
%%configure -f
{ "conf":{
          "spark.pyspark.python": "python3",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv",
          "spark.sql.execution.arrow.enabled":"true"
         }
}

[I 2023-01-16 20:39:54,601.601 configure_magic] Magic cell payload received: {"conf": {"spark.pyspark.python": "python3", "spark.pyspark.virtualenv.enabled": "true", "spark.pyspark.virtualenv.type": "native", "spark.pyspark.virtualenv.bin.path": "/usr/bin/virtualenv", "spark.sql.execution.arrow.enabled": "true"}, "proxyUser": "assumed-role_fdp_blitvin-Isengard"}

[I 2023-01-16 20:39:54,602.602 configure_magic] Sending request to update kernel. Please wait while the kernel will be refreshed.


The kernel is successfully refreshed.

In [6]:
%pip install pyEX

     |████████████████████████████████| 163 kB 26.9 MB/s eta 0:00:01
     |████████████████████████████████| 99 kB 17.4 MB/s eta 0:00:01
  Created wheel for sseclient: filename=sseclient-0.0.27-py3-none-any.whl size=6744 sha256=67ccf0fab12040c3a3d1c2e07c833b6bdb8727d64903d72e65a85af702bb12c3
  Stored in directory: /home/hadoop/.cache/pip/wheels/07/67/7e/96edf627ac746de1a5c5cbb8d59ed960f033b8352dc12c545d
Successfully built sseclient
Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install awswrangler

     |████████████████████████████████| 267 kB 20.6 MB/s eta 0:00:01
     |████████████████████████████████| 73 kB 3.9 MB/s  eta 0:00:01
     |████████████████████████████████| 112 kB 89.0 MB/s eta 0:00:01
     |████████████████████████████████| 132 kB 79.5 MB/s eta 0:00:01
     |████████████████████████████████| 242 kB 103.8 MB/s eta 0:00:01
     |████████████████████████████████| 214 kB 99.6 MB/s eta 0:00:01
     |████████████████████████████████| 51 kB 526 kB/s  eta 0:00:01
     |████████████████████████████████| 43 kB 4.6 MB/s  eta 0:00:01
     |████████████████████████████████| 1.1 MB 80.3 MB/s eta 0:00:01
     |████████████████████████████████| 131 kB 93.9 MB/s eta 0:00:01
     |████████████████████████████████| 41 kB 1.8 MB/s  eta 0:00:01
     |████████████████████████████████| 6.6 MB 80.8 MB/s eta 0:00:01
     |████████████████████████████████| 79 kB 19.7 MB/s eta 0:00:01
     |████████████████████████████████| 49 kB 17.7 MB/s eta 0:00:01
     |████████████████████████████████|

In [116]:
import pyEX as p
import pandas as pd
import numpy as np
#import vectorbt as vbt
import matplotlib.pyplot as plt 
import itertools 
import datetime 
import awswrangler as wr
import vectorbt as vbt
import pyspark.sql.functions as py_f
import pyspark.sql.types as py_t
isSandbox = False
if isSandbox:
    token ='Tpk_02dcd2036e7641b880dd4cbb01fa9c67'
    iex_ver = 'sandbox'
else:
    token ='pk_2e94555e43da4135a6032433c6b18fa5'
    iex_ver = 'stable'
pyEX_cl = p.Client(api_token=token)
timeframe='max'

In [9]:
fx_etf_symbols = ['FXA','FXY','FXE','FXB','UUP','FXC','FXF',]
bitcoin_etf_symbols = ['BITO']
commod_etf_symbols = ['USO','GLD','DBA','DBB','SLV',]
fi_etf_symbols = ['IAGG','AGG','IHY','EMLC','HYG',]
equity_etf_symbols = ['AMJ','AMLP','ARKF','ARKG','ARKK','ARKQ','ARKW','BOTZ','BUG','CGW','CIBR','COPX','CRBN','DRIV','EFV','EMLP','FBT','FDN','FIVG',
'FIW','FTXG','GDX','GNR','GUNR','HACK','IAT','IBB','ICLN','IEO','IFRA','IGE','IGF','IGM','IGV','IHF','IHI','ITB','ITZ','IWD','IWN','IWS','IYT','JETS','KBWB',
                      'KOMP','KRE','KWEB','LCTU','LIT','MLPA',
'MLPX','MOO','NFRA','OIH','PABU','PAVE','PBW','PEJ','PHO','PICK','PPA','QCLN','REET','REMX','ROBO','SCHH','SIL','SKYY','SMH','SOXX','SRVR','TAN','URA','URNM',
                      'VUG','XAR','XBI','XHB','XME','XOP','XSD','QQQ','EWZ','AAXJ','VTI','EWJ','EWA','EWC','MCHI','EWU','EWQ','EWG','FM','EIS']
sectors = ['XLC','XLY','XLP','XLE','XLF','XLV','XLI','XLB','XLRE','XLK','XLU']
bench_symbol=['SPY','QQQ','IVV']
all_symbols =fx_etf_symbols+bitcoin_etf_symbols+commod_etf_symbols+fi_etf_symbols+equity_etf_symbols+bench_symbol+sectors
#all_symbols
columns_to_capture = ['close','open','symbol']
data_folder = 's3://fsidatalake/eod_market_data/etf_trend_following/'
parquet_file_out =f"{data_folder}/etf_market_data.parquet"

In [10]:
def get_full_file_name(data_folder,one_sym):
    return(f"{data_folder}{one_sym}.parquet")
def combine_new_and_stored_data(new_df,file_name,columns_to_capture):
    try:
        old_pd = pd.read_parquet(file_name)
        final_df=pd.concat([old_pd,new_df[columns_to_capture]]).drop_duplicates().sort_index()
    except Exception as l_exc:
        final_df=new_df.drop_duplicates().sort_index()
    return(final_df)
if False:
    md_df = None
    for one_sym in all_symbols:
        try:
            new_df = pyEX_cl.chartDF(symbol=one_sym, timeframe=timeframe)
            file_name = get_full_file_name(data_folder,one_sym)
            final_df = combine_new_and_stored_data(new_df,file_name,columns_to_capture)
            assert final_df['symbol'].values[0]==one_sym, 'STOP!!!'
            assert one_sym in file_name, 'STOP!!!'
            final_df[columns_to_capture].to_parquet(file_name,index=True)
            if md_df is None:
                md_df= spark.createDataFrame(final_df[columns_to_capture].reset_index())
            else:
                md_df=md_df.union(spark.createDataFrame(final_df[columns_to_capture].reset_index()))
        except Exception as l_exc:
            print(f"exc:{one_sym}, {l_exc}")
        #break
    md_df.write\
    .option("header",True)\
    .partitionBy('symbol')\
    .mode("overwrite")\
    .parquet(parquet_file_out)

In [11]:
print(new_df.index.min(),new_df.index.max(),final_df.index.min(),final_df.index.max() )

NameError: name 'new_df' is not defined

In [ ]:
all_symbols_rdd = sc.parallelize(all_symbols) 

In [120]:
class MarketDataGateway():
    def __init__(self):
        self.fx_etf_symbols = ['FXA','FXY','FXE','FXB','UUP','FXC','FXF',]
        self.bitcoin_etf_symbols = ['BITO','BTF','XBTF','BITS','GBTC','BITW','BLOK','BLCN','LEGR','SPBC','BITQ','BKCH','DAPP','BTCFX']
        self.bitcoin_equity_proxy_symbols = ['COIN','MSTR']
        self.crypto_mining_symbols = ['RIOT','CAN','HUT','HIVE','MARA','BTCM','BTBT','BITF']
        self.commod_etf_symbols = ['USO','GLD','DBA','DBB','SLV',]
        self.fi_etf_symbols = ['IAGG','AGG','IHY','EMLC','HYG',]
        self.equity_etf_symbols = ['AMJ','AMLP','ARKF','ARKG','ARKK','ARKQ','ARKW','BOTZ','BUG','CGW','CIBR','COPX','CRBN','DRIV','EFV','EMLP','FBT','FDN','FIVG',
                'FIW','FTXG','GDX','GNR','GUNR','HACK','IAT','IBB','ICLN','IEO','IFRA','IGE','IGF','IGM','IGV','IHF','IHI','ITB','ITZ','IWD','IWN','IWS','IYT','JETS','KBWB',
                      'KOMP','KRE','KWEB','LCTU','LIT','MLPA',
                    'MLPX','MOO','NFRA','OIH','PABU','PAVE','PBW','PEJ','PHO','PICK','PPA','QCLN','REET','REMX','ROBO','SCHH','SIL','SKYY','SMH','SOXX','SRVR','TAN','URA','URNM',
                      'VUG','XAR','XBI','XHB','XME','XOP','XSD','QQQ','EWZ','AAXJ','VTI','EWJ','EWA','EWC','MCHI','EWU','EWQ','EWG','FM','EIS']
        self.sectors = ['XLC','XLY','XLP','XLE','XLF','XLV','XLI','XLB','XLRE','XLK','XLU']
        self.bench_symbol=['SPY','QQQ','IVV']
        self.all_symbols =self.fx_etf_symbols+\
                            self.bitcoin_etf_symbols+\
                            self.commod_etf_symbols+\
                            self.bitcoin_equity_proxy_symbols+\
                            self.crypto_mining_symbols+\
                            self.fi_etf_symbols+\
                            self.equity_etf_symbols+\
                            self.bench_symbol+\
                            self.sectors
        self.all_symbols_df = spark.createDataFrame(pd.DataFrame(self.all_symbols,columns=['symbol']))
        #all_symbols
        self.columns_to_capture = ['close','open','symbol']
        self.data_folder = 's3://fsidatalake/eod_market_data/etf_trend_following/'
        self.parquet_file_out =f"{self.data_folder}/etf_market_data.parquet"
        
    def set_highwatermarks(self):
        self.old_df = spark.read.parquet(self.parquet_file_out)
        self.highwatermark_pd = self.old_df.groupby('symbol').agg(py_f.max("date").alias("max_date"),
                             py_f.min("date").alias("min_date"),
                             (py_f.max("date")-py_f.min("date")).alias("date_length")
                            ).toPandas()

#@py_f.pandas_udf("date timestamp, close double, open double, symbol string", py_f.PandasUDFType.GROUPED_MAP)
@py_f.pandas_udf("symbol string", py_f.PandasUDFType.GROUPED_MAP)
def get_new_data_df(symbol):
    hwm_l=highwatermark_pd_bc.value
    symbol=symbol
    #pyEX_cl = p.Client(api_token=token)
    l_pd= pyEX_cl_bc.chart(symbol=sym, timeframe=timeframe,last=last_days).query(f"date>'{(max_date).strftime('%Y-%m-%d')}'")
    v= vbt.portfolio.SizeType.Value
    #return(l_pd[columns_to_capture_bc.value])
    return(symbol)
        
        
mdg=MarketDataGateway()
mdg.set_highwatermarks()
highwatermark_pd_bc =sc.broadcast(mdg.highwatermark_pd)
columns_to_capture_bc =sc.broadcast(mdg.columns_to_capture)

PicklingError: Can't pickle <function metadata at 0x7f78bc09e9e0>: it's not the same object as pyEX.account.metadata

In [ ]:
mdg.all_symbols_df.groupby("symbol").apply(get_new_data_df).show()

In [30]:
lpd=mdg.highwatermark_pd
sym='ARKF'
max_date = lpd.query(f"symbol=='{sym}'")['max_date'].iloc[0]
last_days=(datetime.datetime.now()-max_date).days
res_pd = pyEX_cl.chartDF(symbol=sym, timeframe=timeframe,last=last_days).query(f"date>'{(max_date).strftime('%Y-%m-%d')}'")


In [117]:
vbt.portfolio.SizeType.Value

1

In [54]:
[('symbol',i) for i in mdg.all_symbols ]

[('symbol', 'FXA'),
 ('symbol', 'FXY'),
 ('symbol', 'FXE'),
 ('symbol', 'FXB'),
 ('symbol', 'UUP'),
 ('symbol', 'FXC'),
 ('symbol', 'FXF'),
 ('symbol', 'BITO'),
 ('symbol', 'BTF'),
 ('symbol', 'XBTF'),
 ('symbol', 'BITS'),
 ('symbol', 'GBTC'),
 ('symbol', 'BITW'),
 ('symbol', 'BLOK'),
 ('symbol', 'BLCN'),
 ('symbol', 'LEGR'),
 ('symbol', 'SPBC'),
 ('symbol', 'BITQ'),
 ('symbol', 'BKCH'),
 ('symbol', 'DAPP'),
 ('symbol', 'BTCFX'),
 ('symbol', 'USO'),
 ('symbol', 'GLD'),
 ('symbol', 'DBA'),
 ('symbol', 'DBB'),
 ('symbol', 'SLV'),
 ('symbol', 'COIN'),
 ('symbol', 'MSTR'),
 ('symbol', 'RIOT'),
 ('symbol', 'CAN'),
 ('symbol', 'HUT'),
 ('symbol', 'HIVE'),
 ('symbol', 'MARA'),
 ('symbol', 'BTCM'),
 ('symbol', 'BTBT'),
 ('symbol', 'BITF'),
 ('symbol', 'IAGG'),
 ('symbol', 'AGG'),
 ('symbol', 'IHY'),
 ('symbol', 'EMLC'),
 ('symbol', 'HYG'),
 ('symbol', 'AMJ'),
 ('symbol', 'AMLP'),
 ('symbol', 'ARKF'),
 ('symbol', 'ARKG'),
 ('symbol', 'ARKK'),
 ('symbol', 'ARKQ'),
 ('symbol', 'ARKW'),
 ('symbol'

In [109]:
(max_date-10).strftime('%Y-%m-%d')

TypeError: Addition/subtraction of integers and integer-arrays with Timestamp is no longer supported.  Instead of adding/subtracting `n`, use `n * obj.freq`

In [100]:
lpd

,symbol,max_date,min_date,date_length
0,IWN,2023-01-13,2006-12-29,5859 days
1,EIS,2023-01-13,2008-05-29,5342 days
2,EMLC,2023-01-13,2010-07-23,4557 days
3,XSD,2023-01-13,2007-03-01,5797 days
4,REMX,2023-01-13,2010-10-28,4460 days
...,...,...,...,...
119,URNM,2023-01-13,2019-12-04,1136 days
120,KOMP,2023-01-13,2018-10-23,1543 days
121,XLC,2023-01-13,2018-06-19,1669 days
122,FIVG,2023-01-13,2019-03-05,1410 days
